In [2]:
from unittest.mock import Mock, patch
from pytest import raises

## patch.object

If we're working directly with an object, we can use `patch.object`

In [3]:
class User:
    def is_birthday(self):
        pass
        
    def greet(self):
        if self.is_birthday():
            return 'happy birthday'
        else:
            return 'hello'

user = User()
with patch.object(user, 'is_birthday', return_value=True):
    # user.is_birthday is a MagicMock that returns True
    print(user.is_birthday, user.is_birthday())
    # Check the outcome
    assert user.greet() == 'happy birthday'

<MagicMock name='is_birthday' id='140470077091736'> True


However, we don't always have direct access to that object.

## patch

In [4]:
import requests

# a MagicMock is returned by the patched method
with patch('requests.get'):
    print(requests.get)

<MagicMock name='get' id='140470077090112'>


In [5]:
# Example of a function that might need patching
import requests

def get_followers(username):
    response = requests.get('https://api.github.com/users/%s' % username)
    return response.json()['followers']

print(get_followers('helenst'))

25


### decorator

In [20]:
# Function decorator
@patch('requests.get')
def test_get_followers(mock_get):
    mock_get.return_value.json.return_value = {'followers': 100}
    assert get_followers('somebody') == 100
    
test_get_followers()  # succeeds

In [7]:
# Class decorator - if all your test methods want to mock the same thing
@patch('requests.get')
class GitHubTest:
    def test_get_followers(mock_get):
        mock_get.return_value.json.return_value = {'followers': 100}
        assert get_followers('somebody') == 100
    
GitHubTest.test_get_followers()

### context manager

In [8]:
with patch('requests.get') as mock_get:
    mock_get.return_value.json.return_value = {'followers': 100}
    assert get_followers('somebody') == 100

### manual patch control

In [9]:
# Use start and stop for more control
patcher = patch('requests.get')
mock_get = patcher.start()
mock_get.return_value.json.return_value = {'followers': 100}
assert get_followers('somebody') == 100
patcher.stop()

## Where to patch?

```python
## file: github_utils

import requests

def get_followers(username):
   response = requests.get('https://api.github.com/users/%s' % username)
   return response.json()['followers']

```

In [15]:
import github_utils

with patch('requests.get') as mock_get:
    mock_get.return_value.json.return_value = {'followers': 100}
    assert github_utils.get_followers('somebody') == 100  # succeeds

Now with a slightly different module.

```python
# file: github_utils2

from requests import get

def get_followers(username):
   response = get('https://api.github.com/users/%s' % username)
   return response.json()['followers']
```

In [16]:
import github_utils2
    
with patch('requests.get') as mock_get:
    mock_get.return_value.json.return_value = {'followers': 100}
    
    with raises(AssertionError):
        assert github_utils2.get_followers('somebody') == 100

In [14]:
import github_utils2

# "ensure that you patch the name used by the system under test" - the python docs
with patch('github_utils2.get') as mock_get:
    mock_get.return_value.json.return_value = {'followers': 100}
    assert github_utils2.get_followers('somebody') == 100  # succeeds